In [121]:
from glob import glob
import os
import pickle
import shutil
from tqdm import tqdm
from IPython.display import Audio 
import IPython
import random
def load_pickle(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

def load_text(filename):
    with open(filename, "r") as f:
        text = f.readlines()
    return text


def dump_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

In [17]:
data = load_text("./Lib.html") + load_text("./index.html") + load_text("./VCTK.html") + load_text("./lib-seen.html")

In [15]:
val = []
i = 11

for d in data:
    if "<strong>" in d and "text-align: left" in d:
        temp = d.split("strong>")
        d = f"{temp[0]}strong>{i}</strong>{temp[-1]}"
        i += 1
    val.append(d)

In [16]:
with open("temp.html", "w") as f:
    for v in val:
        f.write(v)

In [39]:
ava = []
for idx,d in enumerate(data):
    if "/22/" in d:
        w = d.split('/22/')[-1].split('"></audio>')[0]
        t = data[idx-23].split('</strong>.')[-1].split('</td>')[0]
        ava.append([w, t])

In [40]:
ava[:10]

[['4077_13754_000003_000005.wav',
  ' He was unjustly charged with favoring secession; but the charge was soon disproved.'],
 ['8463_294828_000021_000000.wav',
  ' The lad was thirty years old, and his age to that of his employer was as fifteen is to twenty.'],
 ['1580_141084_000082_000000.wav',
  ' Holmes looked triumphantly at Bannister.'],
 ['2830_3979_000017_000003.wav',
  ' We were resolved not to present this entire mass of exegesis.'],
 ['2300_131720_000002_000002.wav',
  ' The London plant was less temporary, but not permanent, supplying before it was torn out no fewer than three thousand lamps in hotels, churches, stores, and dwellings in the vicinity of Holborn Viaduct.'],
 ['121_121726_000031_000001.wav', ' The Scottish National Hymn.'],
 ['1284_1180_000044_000001.wav',
  ' "I want it to bring my Patchwork Girl to life."'],
 ['5639_40744_000004_000008.wav',
  ' You may wonder to hear me speak thus, being so young.'],
 ['121_127105_000010_000000.wav',
  ' I can see Douglas th

In [56]:
wavs = glob("./*/22/*.wav")
ws = [i.split("/")[-1] for i in wavs]


In [57]:
len(wavs)

276

In [59]:
for w in tqdm(wavs):
    k = w.replace("/22/", "/16/")
    if os.path.exists(k):
        os.remove(k)
        os.system("ffmpeg -i "+ w+ " -loglevel fatal -y -ac 1 -ar 16000"+" " + k)

100%|█████████████████████████████████████████| 276/276 [00:09<00:00, 30.63it/s]


In [102]:
speech = "./eval/gt/Singaporean_Mandarin_0/Singaporean_Mandarin_0_106.wav"

print(load_text(speech.replace("/gt/", "/your/").replace(".wav", ".whisper"))[0])

print("gt")
IPython.display.display(Audio(speech))
print("your")
IPython.display.display(Audio(speech.replace("/gt/", "/your/")))
print("usat")
IPython.display.display(Audio(speech.replace("/gt/", "/USAT/")))

 The apple pie is good.
gt


your


usat


In [109]:
top = load_text("./r.txt")
top = [i.replace("\n", "") for i in top]
top

['Bangla_0_115',
 'Arabic_0_77',
 'Cantonese_4_107',
 'Farsi_1_117',
 'Hindi_5_102',
 'Fulfulde_2_59',
 'Korean_4_100',
 'Mandarin_13_104',
 'Vietnamese_1_84',
 'Singaporean_Mandarin_0_106']

In [129]:
wavs = glob("./eval/your/*/*.whisper")

sels = []

while len(sels) < 120:
    t = random.choice(wavs).split("/")[-1].split(".")[0]
    if t not in sels:
        sels.append(t)

In [132]:
c = 10

with open("temp.txt", "w") as f:

    for t in sels:
        n = t.split("_")[-1]
        sid = t.split("_")[-2]
        spk = "_".join(t.split("_")[:-2])
        
        ref_n = random.randint(-5,5) + int(n)
        refa = f"./eval/gt/{spk}_{sid}/{spk}_{sid}_{ref_n}.wav"
        while not os.path.exists(refa) or ref_n == int(n):
            ref_n = random.randint(-5,5) + int(n)
            refa = f"./eval/gt/{spk}_{sid}/{spk}_{sid}_{ref_n}.wav"
            
        print(spk, sid, n)
        text = load_text(f"./eval/your/{spk}_{sid}/{spk}_{sid}_{n}.whisper")[0]
        
        f.write('          <tr>\n')
        f.write(f'            <td style="text-align: left"><strong>{c}</strong>. {text}</td>\n')
        

        shutil.copyfile(f"./eval/gt/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/gt/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/gt/{spk}_{sid}_{n}.wav"></audio>\n')
        
        shutil.copyfile(f"./eval/gt/{spk}_{sid}/{spk}_{sid}_{ref_n}.wav", f"./esltts/ref/{spk}_{sid}_{ref_n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/ref/{spk}_{sid}_{ref_n}.wav"></audio>\n')
        
        shutil.copyfile(f"./eval/metastylespeech/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/mst/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/mst/{spk}_{sid}_{n}.wav"></audio>\n')
        
        shutil.copyfile(f"./eval/stylespeech/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/st/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/st/{spk}_{sid}_{n}.wav"></audio>\n')
        
        
        shutil.copyfile(f"./eval/your/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/your/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/your/{spk}_{sid}_{n}.wav"></audio>\n')
        
        shutil.copyfile(f"./eval/USAT/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/16/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/16/{spk}_{sid}_{n}.wav"></audio>\n')
        
        shutil.copyfile(f"./eval/USAT/{spk}_{sid}/{spk}_{sid}_{n}.wav", f"./esltts/22/{spk}_{sid}_{n}.wav")
        f.write('            <td style="text-align: center">\n')
        f.write(f'              <audio style="width: 100px;" controls="" preload="none" src="esltts/22/{spk}_{sid}_{n}.wav"></audio>\n')
        
        
        f.write('            </td>\n')
        f.write('          </tr>\n')


        c += 1
    

Vietnamese 2 86
Korean 12 138
Korean 7 117
Mandarin 3 132
Spanish_Heritage 2 146
Korean 4 99
Bangla 1 117
Cantonese 12 130
German 1 140
Spanish_Heritage 6 143
Brazilian_Portuguese 0 103
German 1 142
Kanuri 1 61
Hausa 0 55
Cantonese 0 111
Mandarin 17 125
Kanuri 4 106
Hausa 8 74
Spanish_Heritage 2 147
Kanuri 4 108
Mandarin 0 122
Kanuri 8 97
Mandarin 2 121
Kanuri 2 91
Mandarin 8 82
Spanish_Heritage 9 148
Hausa 1 68
Korean 9 111
Japanese 1 110
Korean 10 92
Cantonese 2 106
Mandarin 0 128
Mandarin 7 136
Korean 12 137
Kanuri 8 98
Spanish_Heritage 0 132
Japanese 2 101
Spanish 3 102
Mandarin 8 89
Spanish_Heritage 8 147
Bangla 1 121
Greek 0 128
Mandarin 15 116
Hindi 6 112
Spanish_Heritage 4 142
Hebrew 3 126
German 1 145
Hebrew 3 132
Korean 4 96
Korean 3 126
Korean 4 98
Russian 4 139
Hindi 1 97
Hebrew 3 131
French 0 134
Hindi 7 153
Russian 3 136
German 1 137
Odiya 0 124
Korean 7 120
Hindi 4 141
Cantonese 8 120
Kanuri 1 64
Mandarin 10 93
Korean 1 106
Spanish_Heritage 8 154
Arabic 1 96
Hindi 0 153


In [116]:
load_text("./u.txt")

['          <tr>\n',
 '            <td style="text-align: left"><strong>1</strong>. Such was Catherine Morland at ten.</td>\n',
 '            <td style="text-align: center">\n',
 '              <audio style="width: 100px;" controls="" preload="none" src="lib-seen/gt/19_198_000010_000000.wav"></audio>\n',
 '            </td>\n',
 '            <td style="text-align: center">\n',
 '              <audio style="width: 100px;" controls="" preload="none" src="lib-seen/ref/19_198_000024_000005.wav"></audio>\n',
 '            </td>\n',
 '            <td style="text-align: center">\n',
 '              <audio style="width: 100px;" controls="" preload="none" src="lib-seen/re/19_198_000010_000000.wav"></audio>\n',
 '            </td>\n',
 '            <td style="text-align: center">\n',
 '              <audio style="width: 100px;" controls="" preload="none" src="lib-seen/st/19_198_000010_000000.wav"></audio>\n',
 '            </td>\n',
 '            <td style="text-align: center">\n',
 '           

3

In [ ]:
data = load_text("./Lib.html") + load_text("./index.html") + load_text("./VCTK.html") + load_text("./lib-seen.html")

In [140]:
tar = "./Lib.html"

data = load_text(tar)

with open(tar, "w") as f:
    for i in range(len(data)):
        if "Reconstructed" in data[i]:
            continue
        if "/re/" in data[i] or "/re/" in data[i-1] or (i<len(data)-1 and "/re/" in data[i+1]):
            continue
        f.write(data[i])